In [1]:
import os
import pandas as pd
import mne
from scipy.signal import welch

In [2]:
def bdf_to_csv(bdf_file):
    raw = mne.io.read_raw_bdf(bdf_file, preload=True)
    eeg_data = raw.get_data()
    channel_names = raw.ch_names
    time_index = raw.times
    
    df = pd.DataFrame(data=eeg_data.T, columns=channel_names, index=time_index)
    req_channels = ['O1', 'O2', 'F3', 'F4', 'C3', 'C4', 'Fp1', 'Fp2']
    df = df[req_channels]
    
    segment_size = 1024
    desired_size = 606
    num_segments = len(df) // segment_size
    
    reduced_df = pd.DataFrame(columns=df.columns)
    
    for i in range(num_segments):
        start_idx = i * segment_size
        end_idx = (i + 1) * segment_size
        segment_data = df.iloc[start_idx:end_idx]
        mean_values = segment_data.mean()
        reduced_df = reduced_df.append(mean_values, ignore_index=True)
    reduced_df = reduced_df.iloc[:desired_size]
    
    df = reduced_df

    file_name = os.path.splitext(os.path.basename(bdf_file))[0]
    output_csv_file = f"./eeg_data/{file_name}.csv"
    
    df.to_csv(output_csv_file)

In [3]:
%%capture

data_dir = './raw_data/'

for filename in os.listdir(data_dir):
    if filename.endswith('.bdf'):
        eeg_data = bdf_to_csv(os.path.join(data_dir, filename))

In [4]:
def extract_mean_and_psd_mean(csv_file):
    new_df = pd.DataFrame(columns=['channel', 'mean', 'psd_mean'])
    req_channels = ['O1', 'O2', 'F3', 'F4', 'C3', 'C4', 'Fp1', 'Fp2']
    df = pd.read_csv(csv_file)
    for channel in df.columns:
        if channel not in req_channels:
            continue
        mean_val = df[channel].mean()
        _, psd = welch(df[channel], fs=256)
        new_rec = {
            'channel': channel,
            'mean': mean_val,
            'psd_mean': psd.mean()
        }
        new_df = new_df.append(new_rec, ignore_index=True)
    return new_df

In [5]:
new_data_dir = './eeg_data/'
dest_data_dir = './mean_and_psd/'

In [6]:
%%capture

for csv_file in os.listdir(new_data_dir):
    file = os.path.join(new_data_dir, csv_file)
    df = extract_mean_and_psd_mean(file)
    dest = os.path.join(dest_data_dir, csv_file)
    print(dest)
    df.to_csv(dest)

In [9]:
data_dir = './mean_and_psd/'

def generate_labels(labels_df):
    for file in os.listdir(data_dir):
        task = file.split('_')[1].split('.')[0].split('-')[1]
        label = 0
        if "med1" in task:
            label = 0
        
        elif "med2" in task:
            label = 1
        
        elif "think1" in task:
            label = 2
        
        elif "think2" in task:
            label = 3
        
        task_label = {
            'label': label,
            'task': task
        }
        labels_df = labels_df.append(task_label, ignore_index=True)
        print(task)
    
    return labels_df

In [10]:
labels_df = pd.DataFrame(columns=['label', 'task'])
generate_labels(labels_df)

med1breath
med2
think1
think2
med1breath
med2
think2


C:\Users\Siddarth\AppData\Local\Temp\ipykernel_23168\1582262698.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels_df = labels_df.append(task_label, ignore_index=True)
C:\Users\Siddarth\AppData\Local\Temp\ipykernel_23168\1582262698.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels_df = labels_df.append(task_label, ignore_index=True)
C:\Users\Siddarth\AppData\Local\Temp\ipykernel_23168\1582262698.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels_df = labels_df.append(task_label, ignore_index=True)
C:\Users\Siddarth\AppData\Local\Temp\ipykernel_23168\1582262698.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

,label,task
0,0,med1breath
1,1,med2
2,2,think1
3,3,think2
4,0,med1breath
5,1,med2
6,3,think2
